In [77]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import tensorflow as tf
import os
from tqdm import tqdm

In [72]:
PATH = '/content/drive/MyDrive/Colab Notebooks/VCET/archive/Dataset/'
label_df = pd.read_csv(PATH + "labels.csv")

In [73]:
label_df['Labels'][0]

1

In [74]:
images, labels = [], []
imgList = os.listdir(PATH + 'images/')
for i in tqdm(range(len(imgList))):
        curr = cv2.imread(PATH + 'images/' + str(imgList[i]))
        curr = cv2.cvtColor(curr,cv2.COLOR_BGR2GRAY)
        images.append(curr)
        labels.append(label_df['Labels'][i])

100%|██████████| 441/441 [00:03<00:00, 128.70it/s]


In [75]:
images = np.array(images)
labels = np.array(labels)
print(len(images), len(labels))

441 441


In [98]:
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)

## Train Test Split

In [99]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

## Creating model

In [105]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (5,5), activation= 'relu', input_shape  = (384,512,1)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(32, (5,5), activation= 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(64, (5,5), activation= 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation= 'relu'),
                                    tf.keras.layers.Dense(2, activation= 'softmax')])

In [106]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = RMSprop(learning_rate = 0.001), metrics = ['accuracy'])

In [107]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 380, 508, 16)      416       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 190, 254, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 186, 250, 32)      12832     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 93, 125, 32)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 89, 121, 64)       51264     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 44, 60, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 168960)           

In [108]:
model_fit = model.fit(X_train, y_train, steps_per_epoch = 3, epochs = 30)

Epoch 1/30
3/3 [==============================] - 96s 31s/step - loss: 8753.1211 - accuracy: 0.4351
Epoch 2/30
3/3 [==============================] - 93s 31s/step - loss: 865.1968 - accuracy: 0.5260
Epoch 3/30
3/3 [==============================] - 94s 31s/step - loss: 74.4579 - accuracy: 0.4416
Epoch 4/30
3/3 [==============================] - 93s 31s/step - loss: 0.6766 - accuracy: 0.5779
Epoch 5/30
3/3 [==============================] - 93s 31s/step - loss: 0.6211 - accuracy: 0.7597
Epoch 6/30
3/3 [==============================] - 93s 31s/step - loss: 9.6829 - accuracy: 0.6786
Epoch 7/30
3/3 [==============================] - 93s 31s/step - loss: 4.4995 - accuracy: 0.5974
Epoch 8/30
3/3 [==============================] - 95s 32s/step - loss: 0.5721 - accuracy: 0.7565
Epoch 9/30
3/3 [==============================] - 93s 31s/step - loss: 0.3545 - accuracy: 0.8701
Epoch 10/30
3/3 [==============================] - 93s 31s/step - loss: 0.3467 - accuracy: 0.8539
Epoch 11/30
3/3 [======

In [109]:
score = model.evaluate(X_test,y_test,verbose=0)
print('Test Score : ',score[0])
print('Test Accuracy : ', score[1])

Test Score :  1.505253553390503
Test Accuracy :  0.6390977501869202


In [111]:
model.save(PATH + 'model_v1')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/VCET/archive/Dataset/model_v1/assets


In [113]:
model = tf.keras.models.load_model(PATH + 'model_v1')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 380, 508, 16)      416       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 190, 254, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 186, 250, 32)      12832     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 93, 125, 32)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 89, 121, 64)       51264     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 44, 60, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 168960)           

In [150]:
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/VCET/archive/Dataset/test.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(384,512))
img = img.reshape((1,384,512,1))

In [151]:
p = model.predict(img)
if(p[0][0] < p[0][1]):
  print("No Flood Detected")
else:
  print("Flood Detected")

No Flood Detected


In [152]:
model_fit = model.fit(X_train, y_train, steps_per_epoch = 3, epochs = 15)

Epoch 1/15
3/3 [==============================] - 104s 31s/step - loss: 1035.5297 - accuracy: 0.6721
Epoch 2/15
3/3 [==============================] - 93s 31s/step - loss: 61.1566 - accuracy: 0.5455
Epoch 3/15
3/3 [==============================] - 93s 31s/step - loss: 21.2396 - accuracy: 0.5097
Epoch 4/15
3/3 [==============================] - 93s 31s/step - loss: 4.6851 - accuracy: 0.5649
Epoch 5/15
3/3 [==============================] - 93s 31s/step - loss: 0.6882 - accuracy: 0.6396
Epoch 6/15
3/3 [==============================] - 93s 31s/step - loss: 0.6871 - accuracy: 0.6396
Epoch 7/15
3/3 [==============================] - 93s 31s/step - loss: 0.6966 - accuracy: 0.6396
Epoch 8/15
3/3 [==============================] - 93s 31s/step - loss: 0.6874 - accuracy: 0.6396
Epoch 9/15
3/3 [==============================] - 93s 31s/step - loss: 0.6870 - accuracy: 0.6396
Epoch 10/15
3/3 [==============================] - 93s 31s/step - loss: 0.6864 - accuracy: 0.6396
Epoch 11/15
3/3 [======

In [153]:
score = model.evaluate(X_test,y_test,verbose=0)
print('Test Score : ',score[0])
print('Test Accuracy : ', score[1])

Test Score :  0.6825058460235596
Test Accuracy :  0.6842105388641357
